#### @prompt example

## LangChain with OpenBB
This notebook will leverage the following technologies/frameworks to do some stock research.
It will create an Agent that will fetch information for top performing companies in the top performing industry
It will then leverage Chroma to load the latest earning transcript call for Altria (MO) to allow users to ask few questions
on Altria's latest result.
This work was inspired by this article

- Gemini
- LangChain
- OpenBB (openbb.co)

In [ ]:
!pip install langchain
!pip install -U langchain-google-genai
!pip install -U -q "google-genai==1.7.0"
!pip install langchain_community
!pip install docx2txt
!pip install chromadb
!pip install wikipedia
!pip install finvizfinance

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
from langchain_core.tools import tool

@tool
def get_sectors_performance(sector:str = None):
    """ Useful for getting performance of each  sector for last week, last month, last quarter, last half year and last year. **This tool does not require any input from the user.**"""
    
    from finvizfinance.group import Performance

    try:
        performance = Performance()
        # Get the performance data
        return performance.screener_view().to_dict('records')
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure the finvizfinance library is installed correctly.")
        print("You can install it using: pip install finvizfinance")
        print("Also, check your internet connection as the library fetches data from Finviz.")

@tool
def get_companies_for_sector(sectorName:str):
    """ Return a subset of companies for the given sector"""
    from finvizfinance.screener.overview import Overview
    foverview = Overview()
    filters_dict = {'Sector': sectorName,
                    'Market Cap.': '+Small (over $300mln)',
                    'Average Volume': 'Over 200K',
                    'Current Ratio': 'Over 1',
                    'Debt/Equity': 'Under 1',
                    'InstitutionalOwnership': 'Under 60%',
                    'Price': 'Over $10'}
    foverview.set_filter(filters_dict=filters_dict)
    df = foverview.screener_view(order='Company')
    return df.head(5).to_dict('records')


get_companies_for_sector.invoke('Basic Materials')

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

from langchain.agents import create_structured_chat_agent, AgentExecutor

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain




### Chat Memory

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentType, initialize_agent

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# Initialize Gemini Pro model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)


# Define the tools and create a "tools" node.

tools = [get_sectors_performance, get_companies_for_sector ]

# Attach the tools to the model so that it knows what it can call.
#llm_with_tools = llm.bind_tools(tools)

In [ ]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
# Define your prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, max_token_limit=18000)

# Initialize the agent
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

### Finding the best performing sector across week, month, quarter, and year

In [ ]:
input1 = '''Find the sector which across week, month, quarter and year has shown the best performance and summarize it.
'''
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

### Now we are going to find few companies for the best performing sector. We'll output a table sorted by P/E

In [ ]:
input1 = "Now find me some companies for the sector you found in the previous step. Create  a table with Ticker, Company,  P/E and change"
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

### Now we will play around with Chroma
### We will upload an extract from Altria Group latest Earning calls and ask our llm to extract some information

In [83]:
!pip install -U -q "google-genai==1.7.0"

In [96]:
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import create_retriever_tool

import os

# --- Configure Google API Key (if needed) ---
# os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"

# --- Define the LLM and Embeddings ---
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# --- Load and Index the Text Document ---
def load_document_into_chroma(file_path, persist_directory="chroma_db_single"):
    """Loads a text document, creates embeddings, and stores it in ChromaDB."""
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
    vectorstore.persist()
    return vectorstore

# --- Define the Retrieval Tool ---
def query_chroma_document(query):
    """Queries the ChromaDB for the loaded document."""
    vectorstore = Chroma(persist_directory="chroma_db_single", embedding_function=embeddings)
    retriever = vectorstore.as_retriever()
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

vectorstore = load_document_into_chroma('/kaggle/input/altria-q424-earning-call/Altria_Q424EarningCall.txt')

retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs = {'k' : 3, 'lambda_mult' : 0.7})
retrieval_tool = create_retriever_tool(retriever= retriever,
                                        name="Altria Q424 Earning calls",
                                        description="For any questions regarding Altria latest earning calls, you must use this tool")


# --- Define the Prompt Template for the Agent ---
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that can retrieve information from a specific document based on user queries.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        ("user", "Use the 'Document Retriever' tool to find relevant information in the document."),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# --- Initialize Memory for the Agent ---
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

# --- Initialize the Agent ---
chromaagent = initialize_agent(
    [retrieval_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    prompt=prompt,
    handle_parsing_errors=True
)



### Now we'll query for risk factors and opportunities from earning calls

In [100]:
input1 = '''Please provide the main higlights from latest earning call transcript'''
result = chroma_agent.invoke({"input": input1})
print(result['output'])



> Entering new AgentExecutor chain...
I need to retrieve the highlights from the latest earnings call transcript.
Action: Document Retriever
Action Input: "highlights from latest earnings call transcript"
Observation: quarter, down 8.1% versus the prior year. These earnings reflect the impact of a lower ownership interest compared to the year-ago period due to the partial sale of our ABI investment last year. We continue to view the ABI stake as a financial investment and our goal remains to maximize the long-term value of the investment for our shareholders. We demonstrated our commitment to returning significant value to shareholders and maintaining a strong balance sheet during 2024. For the full year, we returned over $10.2 billion of cash to shareholders through dividends and share repurchases. We paid $6.8 billion in dividends and our board raised our dividend by 4.1% in August, marking our 59th increase in the last 55 years. We also completed our previously authorized share re